In [226]:
import pandas as pd
import os
import dotenv as dotenv
from time import sleep
import sys
import os
# Asegúrate de que ../src esté en el path
sys.path.append(os.path.join(os.path.dirname(os.path.abspath('')), '../src'))
# Intenta importar la función
from dic import diccionario_correspondencia1
from dic import diccionario_correspondencia2
from dic import diccionario_correspondencia3

In [227]:
df_clasidicacion=pd.read_csv('../../data/base/clasificaciones_premier.csv')
df_info=pd.read_csv('../../data/base/info_clubes.csv')
df_estadios=pd.read_csv('../../data/base/asistencia_estadios.csv')
df_jugadores1=pd.read_csv('../../data/base/players_stats_season_2023.csv')
df_jugadores2=pd.read_csv('../../data/base/players_stats_season_2022.csv')
df_jugadores3=pd.read_csv('../../data/base/players_stats_season_2021.csv')
df_jugadores4=pd.read_csv('../../data/base/players_stats_season_2020.csv')
df_jugadores5=pd.read_csv('../../data/base/players_stats_season_2019.csv')
df_jugadores6=pd.read_csv('../../data/base/players_stats_season_2018.csv')
df_jugadores7=pd.read_csv('../../data/base/players_stats_season_2017.csv')

In [228]:
df_jugadores=pd.concat([df_jugadores1, df_jugadores2, df_jugadores3, df_jugadores4,df_jugadores5,df_jugadores6,df_jugadores7], axis=0).reset_index(drop=True)

In [229]:
len(df_jugadores['player_id'].unique())

2225

In [230]:
len(sorted(df_jugadores['team'].unique()))

42

In [231]:
len(sorted(df_clasidicacion['equipo'].unique()))

43

In [232]:
len(sorted(df_info['Nombre Completo'].unique()))

23

In [233]:
len(sorted(df_estadios['Equipo'].unique()))

51

Crear lista de equipos

In [234]:
lista_equipos=sorted(df_estadios['Equipo'].unique())

In [235]:
df_clubes=pd.DataFrame(lista_equipos,columns=['Equipos'])

Homogeneizar para que todos los df tengan la misma columna de Equipo definida

In [236]:
df_clasidicacion['equipo'] = df_clasidicacion['equipo'].map(diccionario_correspondencia1)

In [237]:
df_info['Equipo']=df_info['Nombre Completo'].map(diccionario_correspondencia2)

In [238]:
df_jugadores['team']=df_jugadores['team'].map(diccionario_correspondencia3)

Traducir las columnas del df_jugadores

In [239]:
# Diccionario de traducción de columnas
columnas_traducidas = {
    'season': 'temporada',
    'player_id': 'id_jugador',
    'player_name': 'nombre_jugador',
    'age': 'edad',
    'nationality': 'nacionalidad',
    'team': 'equipo',
    'games_played': 'partidos_jugados',
    'minutes': 'minutos',
    'goals': 'goles',
    'yellow_cards': 'tarjetas_amarillas',
    'red_cards': 'tarjetas_rojas'
}

# Aplicar las traducciones a las columnas de un DataFrame (df)
df_jugadores = df_jugadores.rename(columns=columnas_traducidas)


In [240]:
df_jugadores['edad'] = df_jugadores['edad'] - (2024 - df_jugadores['temporada'])

Borrar jugadores que no se sepa de que equipo son

In [242]:
df_jugadores.shape

(5347, 12)

In [243]:
df_jugadores=df_jugadores.dropna(subset=['equipo'])

In [244]:
df_jugadores.shape

(3365, 12)

In [245]:
df_jugadores=df_jugadores.dropna(subset=['edad'])

In [246]:
df_jugadores.shape

(3364, 12)

In [247]:
df_jugadores=df_jugadores.fillna(0)

Homogeneizar para que todos los df tengan la misma columna de Temporada

In [248]:
df_clasidicacion['temporada'] = df_clasidicacion['temporada'].str.split('-').str[0].astype(int)

In [249]:
# Función para convertir el formato "92/93" a "1992"
def convertir_temporada(temporada):
    # Tomar los primeros dos dígitos del año inicial
    anio_inicio = int(temporada.split('/')[0])
    # Si el año es menor a 50, asumir que es 2000s; si es mayor, es 1900s
    if anio_inicio < 50:
        return 2000 + anio_inicio
    else:
        return 1900 + anio_inicio

In [250]:
df_estadios['Temporada'] = df_estadios['Temporada'].apply(convertir_temporada)

Homogeneizar datos de rendimiento en Estadios

In [251]:
# Limpiar la columna y convertir a float
df_estadios['Máximo rendimiento'] = df_estadios['Máximo rendimiento'].str.replace(',', '.').str.replace('%', '').astype(float)

# Renombrar la columna
df_estadios.rename(columns={'Máximo rendimiento	': 'Máximo rendimiento (%)'}, inplace=True)

Rellenar NaN de lleno en df_estadios

In [252]:
df_estadios.fillna(0, inplace=True)

Rellenar NaN de titulos en df_info

In [253]:
df_info.fillna(0, inplace=True)

In [254]:
df_info.dtypes

Nombre Completo                   object
Año de fundación                   int64
País                              object
Confederación Nacional            object
Colores principales               object
Colores alternativos              object
Apodo                             object
Web oficial                       object
Títulos Internacionales            int64
Copa Mundial de Clubes             int64
UEFA Champions League              int64
UEFA Europa League                 int64
Supercopa Europea                  int64
Recopa de Europa                 float64
Títulos Nacionales                 int64
Liga Inglesa                       int64
FA Cup                             int64
Supercopa Inglesa                  int64
Copa de la Liga de Inglaterra      int64
Full Members' Cup                float64
Equipo                            object
dtype: object

In [255]:
df_info[['Recopa de Europa', "Full Members' Cup"]] = df_info[['Recopa de Europa', "Full Members' Cup"]].astype(int)


Borrar columna de asistencias en df_jugadores

In [256]:
df_jugadores=df_jugadores.drop(columns='assists')

In [257]:
df_jugadores.to_csv(f'../../data/transformed/stats_jugadores_transformed.csv', index=False)
df_info.to_csv(f'../../data/transformed/info_clubes_transformed.csv', index=False)
df_clasidicacion.to_csv(f'../../data/transformed/clasificaciones_premier_transformed.csv', index=False)
df_estadios.to_csv(f'../../data/transformed/asistencia_estadios_transformed.csv', index=False)
df_clubes.to_csv(f'../../data/transformed/clubes.csv', index=False)